In [1]:
#!pip install opendatasets

# Install needed libraries

In [58]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
import PIL 
import opendatasets as od 
from torchvision import datasets, transforms
import os 
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import DataLoader
from torchvision.transforms import ToPILImage
import torch 
import torchvision 
import wandb
from torch import nn 
from torchvision.models import resnet34
from tqdm import tqdm  
import torch.nn.functional as F
from torchmetrics.functional import precision_recall
from torchmetrics import ConfusionMatrix
from torchmetrics import MatthewsCorrCoef, Specificity, F1Score, CohenKappa
from torch.utils.data import TensorDataset
from collections import OrderedDict
import splitfolders
from sklearn.model_selection import train_test_split

In [69]:
print(torch.__version__)

1.11.0


In [42]:
wandb.login()

True

# Download and split data on train and test (Copied from EDA notebook)

In [ ]:
#od.download('https://www.kaggle.com/datasets/alxmamaev/flowers-recognition')

In [55]:
def get_class_distribution(dataset_obj):
    count_dict = {k:0 for k,v in dataset_obj.class_to_idx.items()}
    
    for element in dataset_obj:
        y_lbl = element[1]
        y_lbl = idx2class[y_lbl]
        count_dict[y_lbl] += 1
            
    return count_dict

In [62]:
def get_class_distribution_loaders(dataloader_obj, dataset_obj):
    count_dict = {k:0 for k,v in dataset_obj.class_to_idx.items()}
    
    for _,j in dataloader_obj:
        y_idx = j.item()
        y_lbl = idx2class[y_idx]
        count_dict[str(y_lbl)] += 1
            
    return count_dict

In [59]:
root_dir = "../Task/flowers-recognition/flowers"

Size = 224
flower_transform = transforms.Compose([transforms.Resize((Size,Size)),
                                       transforms.ToTensor(), 
                                       transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
flower_dataset = datasets.ImageFolder(root_dir, transform=flower_transform)

idx2class = {v: k for k, v in flower_dataset.class_to_idx.items()}

print("Distribution of classes: \n", get_class_distribution(flower_dataset))

Distribution of classes: 
 {'daisy': 764, 'dandelion': 1052, 'rose': 784, 'sunflower': 733, 'tulip': 984}


In [63]:
targets = flower_dataset.targets

train_idx, valid_idx= train_test_split(
    np.arange(len(targets)), test_size=0.2, random_state=42, shuffle=True, stratify=targets)

train_sampler = SubsetRandomSampler(train_idx)
val_sampler = SubsetRandomSampler(valid_idx)
train_loader = DataLoader(dataset=flower_dataset, shuffle=False, batch_size=32, sampler=train_sampler)
val_loader = DataLoader(dataset=flower_dataset, shuffle=False, batch_size=32, sampler=val_sampler)

Let's define configuration of training for wandb sync. 

In [43]:
#NUM_BATCHES_TO_LOG = 10
config = dict(
    epochs=20,
    classes=5,
    batch_size=128,
    learning_rate=0.005,
    dataset="Flower recognition",
    architecture="DenseNet")

In [44]:
# Device configuration
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


# Split data using splitfolder lib and apply data augmentation on train set (optional)



In [21]:

#splitfolders.ratio("../Task/flowers-recognition/flowers", output="flowers_",
#    seed=1337, ratio=(.8, .2), group_prefix=None, move=False)

In [22]:
#Parameters for dataset
batch_size = 32
valid_split = 0.2
shuffle_dataset = True
random_seed = 42


# Transformation and augmentation (optional)


In [23]:
#Size = 224
#train = "../Task/flowers_/train"
#val = "../Task/flowers_/val"
#mean = [0.4363, 0.4328, 0.3291]
#std = [0.2129, 0.2075, 0.2038]

#train_transforms = transforms.Compose([
#    transforms.Resize((Size, Size)),
 #   transforms.RandomHorizontalFlip(),
  #  transforms.RandomRotation(10),
   # transforms.ToTensor(),
    #transforms.Normalize(torch.Tensor(mean), torch.Tensor(std))
#])

#valid_transforms = transforms.Compose([
 #   transforms.Resize((Size, Size)),
  #  transforms.ToTensor(),
   # transforms.Normalize(torch.Tensor(mean), torch.Tensor(std))
    
#])

#train_dataset = torchvision.datasets.ImageFolder(root = train, transform = train_transforms)
#valid_dataset = torchvision.datasets.ImageFolder(root = val, transform = valid_transforms)

#train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = 32, shuffle =True)
#valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size = 32, shuffle =True)

In [24]:
len(valid_loader)

28

# Vizualization of samples

In [65]:


train_batch, label_train = next(iter(train_loader))
valid_batch, label_val = next(iter(valid_loader))

def showed_transformed_images(dataset):
    loader = torch.utils.data.DataLoader(dataset, batch_size = 6, shuffle = True)
    batch = next(iter(loader))
    images, labels = batch
    grid = torchvision.utils.make_grid(images, nrow = 3)
    plt.figure(figsize = (11, 11))
    plt.imshow(np.transpose(grid, (1, 2, 0)))
    print('labels: ', labels)

In [51]:
def images_per_class(dataset):
    classes = dataset.classes
    num_classes = len(dataset.classes)
    img_dict = {}
    dataset_size = len(dataset)

    for i in range(num_classes):
        img_dict[classes[i]] = 0

    for i in range(dataset_size):
        img, label = dataset[i]
        img_dict[classes[label]] += 1

    return img_dict

In [53]:
def get_class_distribution(dataset_obj):
    count_dict = {k:0 for k,v in dataset_obj.class_to_idx.items()}
    
    for element in dataset_obj:
        y_lbl = element[1]
        y_lbl = idx2class[y_lbl]
        count_dict[y_lbl] += 1
            
    return count_dict

In [67]:
#Check num of images per class in train set 
train_images = get_class_distribution(train_dataset)
train_images

{'daisy': 611, 'dandelion': 841, 'rose': 627, 'sunflower': 586, 'tulip': 787}

In [68]:
#Check num of images per class in test dataset
valid_images = get_class_distribution(valid_dataset)
valid_images

{'daisy': 153, 'dandelion': 211, 'rose': 157, 'sunflower': 147, 'tulip': 197}

# Model selection (transfer learning)

Inception v3 is an image recognition model that has been shown to attain greater than 78.1% accuracy on the ImageNet dataset. The model is the culmination of many ideas developed by multiple researchers over the years. The model itself is made up of symmetric and asymmetric building blocks, including convolutions, average pooling, max pooling, concatenations, dropouts, and fully connected layers. Batch normalization is used extensively throughout the model and applied to activation inputs. Loss is computed using Softmax.

In [29]:
#Firstly we will try to train iception Inceptionv3 
inception = torchvision.models.inception_v3(pretrained=True, aux_logits=False)
for param in inception.parameters():
    param.requires_grad = False

# Change the classification layers
inception.fc = nn.Linear(in_features=2048, out_features=5, bias=True)
inception

Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stri

# EfficientNet

EfficientNet is a convolutional neural network architecture and scaling method that uniformly scales all dimensions of depth/width/resolution using a compound coefficient. Unlike conventional practice that arbitrary scales these factors, the EfficientNet scaling method uniformly scales network width, depth, and resolution with a set of fixed scaling coefficients. It has only 7.8 mln parameters and showed 79.2% Top 1 accuracy on ImageNet. 

In [30]:
efficient_net = torchvision.models.efficientnet_b1(pretrained=True, aux_logits=False)
for param in efficient_net.parameters():
    param.requires_grad = False

# Change the classification layers
efficient_net.classifier[1] = nn.Linear(in_features=1280, out_features=5, bias=True)
efficient_net

EfficientNet(
  (features): Sequential(
    (0): ConvNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): ConvNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): ConvNormActivation(
 

Since both models show appreciate result in image classification tasks I decided to compare their efficiency in Flowers Recognition task 

Lets define some helper functions for training pipeline

In [31]:
def model_pipeline(hyperparameters, train_loader, test_loader, model):
    with wandb.init(project="Provectus_classification_flowers", config=hyperparameters):
        config = wandb.config
        model, criterion, optimizer = make(config, model)
        print(model)
        train(model, train_loader, criterion, optimizer, config)
        return model 

As was shown in EDA notebook dataset is insignificantly imbalanced. For that reason as the main performance metrics 
I used Matthews correlation coefficient and Cohen Kappa. 

I train model on 20 epochs

In [32]:
def make(config, model):
    # Make the model
    model = model.to(device)

    # Make the loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(
        model.parameters(), lr=config.learning_rate)
    
    return model, criterion, optimizer

In [40]:
def train(model, loader, criterion, optimizer, config):
    # Tell wandb to watch what the model gets up to: gradients, weights, and more!
    wandb.watch(model, criterion, log="all", log_freq=10)

    # Run training and track with wandb
    total_batches = len(loader) * config.epochs
    example_ct = 0  # number of examples seen
    batch_ct = 0
    max_acc=0
    train_acc = []
    val_acc = []
    for epoch in tqdm(range(config.epochs)):
        train_loss = 0
        val_loss = 0
        acc = 0.0
        print("Training....")
        model.train()
        matthews_corrcoef = MatthewsCorrCoef(num_classes=5)
        f1 = F1Score(num_classes=5)
        cohenkappa = CohenKappa(num_classes=5)
        for batch_num, (images, labels) in enumerate(loader):

            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            # Forward pass 
            outputs = model.forward(images)
            op = F.softmax(outputs,dim=1)
            final_op = torch.argmax(op,dim=1)
            acc += torch.sum(final_op==labels).item()/len(labels)
            loss = criterion(outputs, labels)
            precision, recall = precision_recall(outputs, labels, average = 'macro', num_classes = 5)
            # Backward pass 
            loss.backward()
            wandb.log({'precision' : precision, 'recall' :recall})
            # Step with optimizer
            optimizer.step()
            train_loss+=(loss.item()/len(images))
            if batch_num%50 ==0 and batch_num!=0:
                    print("TARGET: ",labels)
                    print("OUTPUT: ",final_op)
                    print("Accuracy after ",batch_num,"steps: ",acc/batch_num)
                    print('Precision, recall = ', precision, recall)
            acc_ = acc/len(train_loader)
            train_acc.append(acc)
            mat = matthews_corrcoef(outputs, labels)
            f1_score = f1(outputs, labels)
            ck_train = cohenkappa(outputs, labels)
        print("Epoch: ",epoch,"Loss: ",train_loss," Accuracy: ",acc, "Matthews corr.coef: ", mat, "Cohen Kappa metric on train", ck_train)
        example_ct +=  len(images)
        batch_ct += 1

        # Report metrics every 50th batch
        if ((batch_ct + 1) % 50) == 0:
            train_log(loss, example_ct, epoch, acc)
            
        accuracy_test = 0
        eval_accuracy = 0
        with torch.no_grad():
            model.eval()
            correct_test = 0
            total_test = 0
            print("Validating.....")

            for batch in valid_loader:
                inp,target = batch[0].to(device),batch[1].to(device)
                op = F.softmax(inception.forward(inp))
                final_op = torch.argmax(op,dim=1)
                eval_accuracy += np.sum(final_op.detach().cpu().numpy()==target.detach().cpu().numpy())/len(target)
        accuracy_test = (eval_accuracy/len(valid_loader))
        print("Validation accuracy: ",accuracy_test)
        val_acc.append(accuracy_test)
            
        wandb.log({"Epochs" : epoch, "train_loss": train_loss, "Train accuracy" : train_acc[-1],
                   'Validation accuracy' : accuracy_test,
                  "Matthews corr.coef: ": mat, "F1 score_train: " : f1_score,
                  "Cohen Kappa" : ck_train})
        torch.onnx.export(model, images, "model.onnx")
        wandb.save(f"model_{epoch}.onnx")
        

In [39]:
def train_log(loss, example_ct, epoch, accuracy):
    wandb.log({"epoch": epoch, "loss": loss, 'accuracy' : accuracy}, step=example_ct)
    #print(f"Loss after " + str(example_ct).zfill(5) + f" examples: {loss:.3f}" + f' accuracy: {accuracy}')

In [38]:
config = dict(
    epochs=20,
    classes=5,
    batch_size=128,
    learning_rate=0.005,
    dataset="Flower recognition",
    architecture="InceptionV3")

In [28]:
model_incp = model_pipeline(config, train_loader, valid_loader, inception)

Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stri

  0%|          | 0/20 [00:00<?, ?it/s]

Training....
TARGET:  tensor([3, 2, 1, 4, 0, 4, 4, 1, 0, 3, 4, 4, 4, 4, 2, 1, 1, 2, 1, 0, 1, 4, 0, 4,
        4, 0, 4, 1, 3, 4, 3, 0])
OUTPUT:  tensor([3, 4, 1, 3, 4, 4, 4, 1, 3, 3, 3, 4, 3, 4, 2, 1, 1, 2, 1, 0, 1, 4, 0, 3,
        1, 3, 3, 1, 3, 3, 3, 0])
Accuracy after  50 steps:  0.57875
Precision, recall =  tensor(0.7845) tensor(0.7167)
TARGET:  tensor([1, 3, 0, 3, 1, 0, 1, 0, 3, 2, 4, 2, 2, 0, 2, 4, 3, 2, 1, 3, 0, 3, 3, 4,
        2, 3, 1, 4, 4, 4, 2, 1])
OUTPUT:  tensor([1, 3, 3, 3, 1, 0, 1, 0, 3, 2, 4, 2, 2, 0, 2, 4, 1, 0, 1, 3, 0, 4, 2, 4,
        2, 3, 1, 4, 2, 0, 2, 1])
Accuracy after  100 steps:  0.6475
Precision, recall =  tensor(0.7814) tensor(0.7898)
Epoch:  0 Loss:  3.2503461433308467  Accuracy:  69.51339285714286 Matthews corr.coef:  tensor(0.5166) Cohen Kappa metric on train tensor(0.5040)
Validating.....


C:\Users\USER\AppData\Local\Temp\ipykernel_4356\3234579611.py:77: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  op = F.softmax(inception.forward(inp))


Validation accuracy:  0.8080357142857143


...

Training....
TARGET:  tensor([2, 1, 1, 4, 2, 4, 2, 1, 2, 0, 1, 1, 3, 4, 0, 4, 1, 3, 1, 2, 1, 4, 3, 4,
        4, 4, 2, 0, 1, 1, 1, 1])
OUTPUT:  tensor([2, 1, 2, 2, 2, 4, 2, 1, 2, 0, 1, 1, 3, 4, 0, 4, 4, 0, 0, 2, 1, 4, 3, 4,
        4, 4, 3, 0, 0, 1, 4, 1])
Accuracy after  50 steps:  0.77375
Precision, recall =  tensor(0.7317) tensor(0.7917)
TARGET:  tensor([2, 1, 0, 4, 2, 0, 0, 2, 4, 0, 2, 3, 3, 1, 0, 2, 2, 1, 0, 2, 0, 4, 1, 3,
        0, 4, 1, 3, 4, 1, 1, 3])
OUTPUT:  tensor([3, 3, 0, 4, 2, 3, 1, 2, 4, 2, 1, 3, 3, 1, 0, 0, 4, 4, 0, 2, 0, 4, 1, 3,
        0, 4, 1, 3, 2, 1, 1, 3])
Accuracy after  100 steps:  0.775
Precision, recall =  tensor(0.6879) tensor(0.7136)
Epoch:  18 Loss:  2.9973019464606687  Accuracy:  82.42410714285714 Matthews corr.coef:  tensor(0.5345) Cohen Kappa metric on train tensor(0.5164)
Validating.....
Validation accuracy:  0.8470982142857143


 95%|█████████▌| 19/20 [1:19:29<04:11, 251.44s/it]

Training....
TARGET:  tensor([2, 0, 0, 0, 2, 4, 4, 4, 4, 4, 0, 1, 4, 1, 0, 0, 2, 0, 3, 4, 1, 3, 1, 4,
        0, 4, 2, 2, 3, 0, 3, 1])
OUTPUT:  tensor([2, 0, 0, 3, 3, 4, 4, 4, 3, 1, 0, 1, 4, 1, 0, 0, 2, 0, 3, 4, 1, 3, 1, 4,
        0, 2, 2, 2, 3, 0, 3, 1])
Accuracy after  50 steps:  0.783125
Precision, recall =  tensor(0.8410) tensor(0.8711)
TARGET:  tensor([1, 3, 4, 2, 2, 4, 0, 2, 1, 3, 0, 1, 0, 4, 1, 0, 1, 1, 4, 3, 3, 4, 2, 2,
        1, 0, 3, 4, 3, 3, 4, 1])
OUTPUT:  tensor([1, 3, 4, 2, 2, 4, 1, 2, 3, 1, 0, 1, 2, 3, 1, 0, 1, 1, 2, 3, 3, 4, 4, 2,
        1, 3, 3, 4, 2, 3, 4, 1])
Accuracy after  100 steps:  0.76625
Precision, recall =  tensor(0.7615) tensor(0.7007)
Epoch:  19 Loss:  3.1056488962577924  Accuracy:  81.79017857142857 Matthews corr.coef:  tensor(0.4596) Cohen Kappa metric on train tensor(0.4483)
Validating.....
Validation accuracy:  0.828125


100%|██████████| 20/20 [1:23:41<00:00, 251.07s/it]


Cohen Kappa,▂▇▄▃▄▄▇▆▆▅▆▅█▆▆▄▃▆▂▁
Epochs,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
F1 score_train:,▂▇▄▃▄▄▆▆▆▅▆▅█▆▆▄▃▆▃▁
Matthews corr.coef:,▂▇▄▃▄▄▇▆▆▅▆▅█▆▆▄▃▆▂▁
Train accuracy,▁▆▆▇▇▇▇▇▆▇▇▇▇▇▇▇█▇██
Validation accuracy,▄▁▄▂▃▆▂▅▆▇▂█▇▄▇▇▅▇█▆
precision,▂▄▇▁▇▅▅▅▆▆▄▄▆▆▆▇▄▄▃▁▇▄▄▄▆▅▆▄▅▄█▆▇▄▇█▄▇▃▇
recall,▂▄▇▂▄▅▄▄▅▆▅▁▅▅▆▆▅▂▅▂▆▂▃▂▅▅▅▃▇▃█▅▆▄▅█▅█▃▇
train_loss,▇▁▂▂▂▄▅▄▇▄▇▇▅█▆▆▅█▅▆
Cohen Kappa,0.44828
Epochs,19


In [29]:
config = dict(
    epochs=20,
    classes=5,
    batch_size=128,
    learning_rate=0.005,
    dataset="Flower recognition",
    architecture="EfficientNet")

In [30]:
model_efficient = model_pipeline(config, train_loader, valid_loader, efficient_net)

EfficientNet(
  (features): Sequential(
    (0): ConvNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): ConvNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): ConvNormActivation(
 

  0%|          | 0/20 [00:00<?, ?it/s]

Training....
TARGET:  tensor([3, 4, 1, 1, 2, 4, 0, 0, 4, 0, 0, 2, 3, 0, 4, 2, 1, 4, 0, 3, 1, 1, 1, 0,
        4, 0, 4, 0, 2, 2, 2, 0])
OUTPUT:  tensor([3, 4, 1, 1, 2, 3, 0, 1, 4, 3, 0, 2, 3, 1, 4, 2, 1, 3, 0, 3, 1, 1, 1, 0,
        4, 0, 4, 0, 2, 2, 1, 1])
Accuracy after  50 steps:  0.750625
Precision, recall =  tensor(0.8200) tensor(0.8295)
TARGET:  tensor([1, 3, 1, 2, 0, 4, 0, 2, 2, 0, 1, 1, 1, 4, 4, 4, 4, 2, 1, 1, 1, 1, 2, 1,
        3, 4, 0, 4, 4, 4, 0, 1])
OUTPUT:  tensor([1, 3, 1, 2, 3, 4, 3, 2, 3, 0, 1, 1, 1, 0, 4, 4, 4, 2, 1, 1, 3, 1, 2, 1,
        3, 4, 0, 0, 4, 2, 0, 1])
Accuracy after  100 steps:  0.7828125
Precision, recall =  tensor(0.7467) tensor(0.7952)
Epoch:  0 Loss:  2.0232686568051577  Accuracy:  84.01785714285714 Matthews corr.coef:  tensor(0.8696) Cohen Kappa metric on train tensor(0.8623)
Validating.....


C:\Users\USER\AppData\Local\Temp\ipykernel_4356\3234579611.py:77: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  op = F.softmax(inception.forward(inp))


Validation accuracy:  0.828125


...

Training....
TARGET:  tensor([1, 1, 3, 4, 4, 0, 4, 0, 1, 2, 2, 4, 1, 2, 4, 4, 0, 1, 1, 2, 4, 4, 0, 4,
        3, 1, 3, 4, 2, 4, 4, 2])
OUTPUT:  tensor([1, 1, 3, 4, 4, 0, 1, 0, 1, 2, 2, 4, 1, 2, 4, 4, 0, 1, 1, 2, 4, 4, 0, 4,
        3, 1, 3, 4, 2, 4, 4, 2])
Accuracy after  50 steps:  0.898125
Precision, recall =  tensor(0.9750) tensor(0.9833)
TARGET:  tensor([1, 4, 4, 1, 4, 1, 3, 2, 0, 4, 0, 1, 4, 2, 1, 2, 1, 2, 1, 1, 3, 0, 3, 4,
        1, 1, 3, 4, 4, 0, 2, 2])
OUTPUT:  tensor([1, 4, 4, 1, 4, 1, 3, 2, 0, 2, 0, 1, 4, 2, 1, 2, 4, 2, 1, 1, 3, 0, 3, 4,
        0, 1, 3, 4, 4, 0, 2, 2])
Accuracy after  100 steps:  0.891875
Precision, recall =  tensor(0.9064) tensor(0.9350)
Epoch:  18 Loss:  1.3480417372193187  Accuracy:  95.04464285714286 Matthews corr.coef:  tensor(0.8250) Cohen Kappa metric on train tensor(0.8196)
Validating.....
Validation accuracy:  0.828125


 95%|█████████▌| 19/20 [1:44:39<05:30, 330.74s/it]

Training....
TARGET:  tensor([4, 2, 1, 2, 0, 1, 1, 1, 4, 1, 4, 2, 1, 1, 4, 2, 2, 3, 4, 3, 2, 4, 2, 4,
        1, 1, 2, 0, 0, 0, 2, 3])
OUTPUT:  tensor([4, 2, 1, 2, 0, 1, 1, 1, 4, 1, 4, 2, 1, 1, 4, 2, 2, 3, 4, 3, 2, 4, 2, 4,
        1, 1, 2, 0, 0, 2, 0, 3])
Accuracy after  50 steps:  0.89875
Precision, recall =  tensor(0.9278) tensor(0.9278)
TARGET:  tensor([0, 2, 1, 1, 4, 1, 2, 4, 1, 2, 3, 3, 4, 3, 2, 1, 1, 4, 2, 1, 2, 0, 3, 0,
        0, 3, 4, 2, 1, 0, 1, 3])
OUTPUT:  tensor([0, 4, 1, 1, 4, 1, 2, 4, 1, 2, 3, 3, 4, 3, 0, 1, 1, 4, 2, 1, 2, 4, 3, 1,
        1, 3, 4, 2, 1, 0, 1, 3])
Accuracy after  100 steps:  0.8846875
Precision, recall =  tensor(0.8398) tensor(0.8229)
Epoch:  19 Loss:  1.2613357033613803  Accuracy:  94.70535714285714 Matthews corr.coef:  tensor(0.8634) Cohen Kappa metric on train tensor(0.8605)
Validating.....
Validation accuracy:  0.828125


100%|██████████| 20/20 [1:50:11<00:00, 330.57s/it]


Cohen Kappa,▆▄▁▅▃▆▆▇▄▆▄█▆██▁▇▅▅▆
Epochs,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
F1 score_train:,▆▄▁▅▃▆▆▇▄▆▄█▆██▁▇▅▅▆
Matthews corr.coef:,▆▄▁▅▃▆▆▇▄▆▄█▆██▁▇▅▅▆
Train accuracy,▁▆▆▆▆▆██▇▇▇█▇█▇█▇███
Validation accuracy,██▁██▁█▁████████▁███
precision,▄▆▃▄▅▅▇█▄▆▃▆▆▅▅▃▅▇▆▆█▆▆▅▂▄▂▇▇▆▅▇▆▅▇▄█▆▁█
recall,▁▅▄▂▃▄██▄▆▁▃▅▆▂▅▃▆▆▆█▃▅▄▂▂▂▆▇▅▂▆▄▄▆▅█▅▃▇
train_loss,█▃▃▃▃▃▂▁▂▃▂▂▂▁▁▂▂▂▃▂
Cohen Kappa,0.86047
Epochs,19
